
# Using Transaction and Item Data and prepare for Recommender system

From the previous notebook we used transaction and customer data to predict wether a customer is likely to purchase in next 7 days. That was part1 of our problem. Now we are trying to prepare our data for a recommendar system using Factorization machine (in built algo on Sagemaker)
The data set is huge so we would need to filter relevant data. Some considerations are:
- How often the customer purchases
- How popular is the product. 
- based on this we need to create filtered data set for our recommender model
This notebook will show how we prepare the data for our recommender system

In [26]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import time

import boto3
#import sagemaker
#import sagemaker.amazon.common as smac

from scipy.sparse import csr_matrix, hstack, save_npz, load_npz
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 50)
pd.set_option('max_colwidth', 50)  # default is 50

In [27]:
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)

numpy version: 1.20.3
pandas version: 1.0.1


### Read Dataset

In [28]:
import sagemaker
import boto3
import io

In [29]:
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
bucket

'sagemaker-ap-south-1-659144925604'

In [30]:
s3_client = boto3.client('s3')

In [31]:

file_key = 'kaggle/transactions_train.csv'
obj = s3_client.get_object(Bucket=bucket, Key=file_key)
df = pd.read_csv(io.BytesIO(obj['Body'].read()))
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [32]:
df['t_dat'] = pd.to_datetime(df['t_dat'])

In [33]:
df['t_dat'].describe()

count                31788324
unique                    734
top       2019-09-28 00:00:00
freq                   198622
first     2018-09-20 00:00:00
last      2020-09-22 00:00:00
Name: t_dat, dtype: object

Lets focus on more recent data. Filter for dates starting 2020

In [34]:
df[(df.t_dat >= pd.to_datetime("2020-01-1")) ]['t_dat'].describe()

count                10980132
unique                    266
top       2020-04-11 00:00:00
freq                   162799
first     2020-01-01 00:00:00
last      2020-09-22 00:00:00
Name: t_dat, dtype: object

We have data spanning around 2 years. But lets focus on data from year 2020. Thought process is recent data is more relevant in the context.

In [35]:
df = df[(df.t_dat >= pd.to_datetime("2020-01-1")) ].reset_index(drop=True)

Since customer_id is String, we need to convert it to number. We can create mapping between customer_id and index in the data frame and save it for future reference

In [36]:
df.rename(columns = {'customer_id':'customer_id_encoded'}, inplace = True)
tx_user = pd.DataFrame(df['customer_id_encoded'].unique())
tx_user.columns = ['customer_id_encoded']
tx_user['customer_id'] = tx_user.index+1
tx_customer_index_mapping=tx_user[['customer_id','customer_id_encoded']]
tx_customer_index_mapping.to_csv('customer_mapping_recommender.csv')
df_rank = pd.merge(df,tx_customer_index_mapping,on='customer_id_encoded',how='left')
df_rank=df_rank.drop('customer_id_encoded',axis=1)

In [37]:
df_rank.head()

,t_dat,article_id,price,sales_channel_id,customer_id
0,2020-01-01,835247001,0.033881,2,1
1,2020-01-01,802930002,0.067780,2,2
2,2020-01-01,760084008,0.025407,2,2
3,2020-01-01,804662002,0.033881,2,3
4,2020-01-01,801554002,0.016932,2,3


## Lets select those customers who have done more than 10 transactions
Lets group customer Id with their purchase counts

In [38]:
df_rank_cust = df_rank.groupby(["customer_id"]).size().reset_index()
df_rank_cust.rename(columns = {0:'custTotal'}, inplace = True)
df_rank_cust.head()

,customer_id,custTotal
0,1,8
1,2,23
2,3,6
3,4,2
4,5,27


In [39]:
df_rank_cust.custTotal.value_counts(bins=20)

(0.21, 40.45]      813653
(40.45, 79.9]       39107
(79.9, 119.35]       6997
(119.35, 158.8]      1798
(158.8, 198.25]       642
(198.25, 237.7]       266
(237.7, 277.15]       117
(277.15, 316.6]        57
(316.6, 356.05]        26
(356.05, 395.5]        23
(434.95, 474.4]        10
(395.5, 434.95]         9
(474.4, 513.85]         6
(513.85, 553.3]         5
(553.3, 592.75]         3
(592.75, 632.2]         2
(711.1, 750.55]         1
(632.2, 671.65]         1
(750.55, 790.0]         1
(671.65, 711.1]         0
Name: custTotal, dtype: int64

#### We will consider only those customers who have done atleast 10 transactions since 2021

In [40]:
df_rank_cust = df_rank_cust.query("custTotal>10")

In [41]:
df_filtered_custId= df_rank_cust['customer_id'].to_numpy()
df_new_cust=df_rank[df_rank.customer_id.isin(df_filtered_custId)]

In [42]:
print(df_rank.shape)
print(df_new_cust.shape)

(10980132, 5)
(8581191, 5)


## Lets select those products which have been bought atleast 1000 times
Lets filter on basis of articles being purchased

In [43]:
df_rank_art=df_new_cust.groupby(["article_id"]).size().reset_index()

df_rank_art.rename(columns = {0:'artTotal'}, inplace = True)
df_rank_art.head()

,article_id,artTotal
0,108775015,29
1,108775044,269
2,110065001,37
3,110065002,27
4,110065011,47


In [44]:
df_rank_art.query("artTotal>1000").article_id.nunique()

1684

In [45]:
df_filtered = df_rank_art.query("artTotal>1000")

In [66]:
df_filtered

,article_id,artTotal
5,111565001,1038
6,111586001,1977
7,111593001,1429
14,123173001,1132
33,148033001,1421
...,...,...
58301,918292011,1193
58325,918522001,1158
58328,918547001,1090
58736,924243001,1294


In [65]:
popular_items=df_filtered.sort_values("artTotal", axis=0, ascending=False).reset_index()
popular_items.to_csv("popular_items.csv")

In [47]:
df_filtered_artId= df_filtered['article_id'].to_numpy()

here we are filtering our dataset to include only the popular items which was calculated in previous steps

In [48]:
df_new=df_rank[df_rank.article_id.isin(df_filtered_artId)]

In [49]:
print(df_rank.size)
print(df_new.size)

54900660
19952895


In [50]:
# print(df_rank.size)
# print(df_new.size)

## Feature creation - Determine the rank or rating for customer product combination by interpolating the total purchases of the product as a rank
#### Lets check the filtered data set

In [51]:

df_rank=df_new.groupby(["customer_id","article_id"]).size().reset_index()
#temp = df.groupby(["customer_id","product_id", "product_title", "product_category"]).nunique()["review_id"]
df_rank.tail()

,customer_id,article_id,0
3347122,862714,677930037,1
3347123,862715,568601006,1
3347124,862715,850917001,1
3347125,862717,756322001,1
3347126,862724,740519002,1


In [52]:
df_rank.describe()

,customer_id,article_id,0
count,3.347127e+06,3.347127e+06,3.347127e+06
mean,3.282010e+05,7.505909e+08,1.192240e+00
std,2.274252e+05,1.312542e+08,5.773513e-01
min,1.000000e+00,1.115650e+08,1.000000e+00
25%,1.303970e+05,7.060160e+08,1.000000e+00
50%,2.916750e+05,8.002450e+08,1.000000e+00
75%,5.053780e+05,8.350080e+08,1.000000e+00
max,8.627240e+05,9.337060e+08,8.000000e+01


In [53]:
df_rank.rename(columns = {0:'total'}, inplace = True)
df_rank.head()

,customer_id,article_id,total
0,1,537116001,1
1,2,688558001,3
2,2,706016015,1
3,2,760084008,1
4,2,816563004,2


#### We can run KMeans on the total and cluster them in 5 clusters. We will pivot the cluster number to the rank

In [54]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
kmeans.fit(df_rank[['total']])
df_rank['totalCluster'] = kmeans.predict(df_rank[['total']])

In [55]:
df_rank.totalCluster.value_counts()

0    2841369
1     420019
2      76459
3       8995
4        285
Name: totalCluster, dtype: int64

In [56]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final
df_rank = order_cluster('totalCluster', 'total',df_rank,True)

#### The cluster numbers are 0 to 4. We can manipulate the ranks for rank 1 to 5

In [57]:
df_rank.totalCluster=df_rank['totalCluster']+1

In [58]:
df_rank.rename(columns = {'totalCluster':'rating'}, inplace = True)


In [59]:
# Example of customer with more than one review and rating for a particular product.
df_rank=df_rank.drop('total',axis=1)
df_rank.head(20)

,customer_id,article_id,rating
0,1,537116001,1
1,2,706016015,1
2,2,760084008,1
3,6,566140001,1
4,6,595696005,1
5,6,783346001,1
6,6,787946002,1
7,7,616920001,1
8,7,801512004,1
9,8,812464001,1


## Lets also consider the product data and append the features of the product as part of Sparse matrix

In [60]:
# Save the dataframe to csv.


file_key_art = 'kaggle/articles.csv'
obj_art = s3_client.get_object(Bucket=bucket, Key=file_key_art)
df_articles = pd.read_csv(io.BytesIO(obj_art['Body'].read()))
df_articles.head()

#df_rank.to_csv("fm_preprocessed_filtered.csv", index=False)



,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,4,Dark,5,Black,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,3,Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,1,Dusty Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,4,Dark,5,Black,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,3,Light,9,White,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [61]:
df_articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,4,Dark,5,Black,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,3,Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,1,Dusty Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,4,Dark,5,Black,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,3,Light,9,White,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


#### Lets take columns like product group name (categorical) and detail_desc

In [62]:
columnsretain=['article_id','product_group_name','detail_desc']
df_articles=df_articles[columnsretain]
df_rank_prod=pd.merge(df_rank,df_articles,on="article_id",how="left").reset_index()
df_rank_prod["detail_desc"].fillna("", inplace=True)

In [63]:
df_rank_prod.isna().sum()

index                 0
customer_id           0
article_id            0
rating                0
product_group_name    0
detail_desc           0
dtype: int64

#### Save the filtered data in csv. This can directly be consumed as input dataset by our recommendation model

In [64]:
df_rank_prod.to_csv("fm_preprocessed_filtered_with_attributes.csv", index=False)

In this notebook we used the transaction data after filtering with frequent customers and popular items to create a customer product grouping and manipulated a rating for each of the unique set. We clubbed item metadata to this data as well. This inoput data can be easily used to create a Sparse Dataset which is the required input for creating a Recommender system using Factorization machine. The Rating is our target column.

### For next step move to this __[notebook](https://github.com/vivekghi/customer-purchase-prediction-and-recommendation/blob/main/3-Customer-recommender-FM-with-attributes.ipynb)__